In [1]:
import pandas as pd
import Sastrawi
import googletrans
import re

In [4]:
data=pd.read_excel('D:/Skripsi/Data/bjorka_new.xlsx')
del data['Unnamed: 0']
data

,created at,username,text
0,2022-11-02,rudiartraa,"yaah , udah selesai nih Bjorka Bjorkaan nya ?"
1,2022-11-02,oi_filix315,"Kadang gua mikir sekilas pp gua mirip bjorka, ..."
2,2022-11-01,falahsanjaya_,Knp ga ada yg jadi Bjorka Halloween party kema...
3,2022-11-01,namanyaajess,"Beritanya taik, mau ferdi sambo, mau bjorka, m..."
4,2022-11-01,namakubaim,Apa kabar Bjorka?
...,...,...,...
19995,2022-09-11,SecretAdmirerOP,hari ini ada 2 hari bersejarah\n9/11\nbjorka s...
19996,2022-09-11,yonforger,"lah anjir aku baru tau bjorka tu hacker, kirai..."
19997,2022-09-11,semangkalucu,kata w mending bjorka ngewarin tiket the link aja
19998,2022-09-11,choaciao,Bang bjorka bisa ga si hek hati doi biar sama ...


In [8]:
import re
import numpy as np

In [9]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory=StemmerFactory()
stemmer=factory.create_stemmer()

In [10]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

In [12]:
kamus_alay=pd.read_csv('D:/Skripsi/Data/new_kamusalay.csv',header=None,encoding='latin-1')
kamus_alay=kamus_alay.rename(columns={0:'asli',1:'baru'})
kamus_alay

,asli,baru
0,anakjakartaasikasik,anak jakarta asyik asyik
1,pakcikdahtua,pak cik sudah tua
2,pakcikmudalagi,pak cik muda lagi
3,t3tapjokowi,tetap jokowi
4,3x,tiga kali
...,...,...
15162,mendikbud,menteri pendidikan dan kebudayaan
15163,mendag,menteri perdagangan
15164,menaker,menteri tenaga kerja
15165,memetwit,mentwit


In [13]:
#Lower text
def lowercase(text):
    return text.lower()

In [14]:
#Remove non alphanumeric character
def removeNonAlphanumeric(text):
    text = re.sub('[^0-9a-zA-Z]+', ' ', text) 
    text="".join(filter(lambda x: not x.isdigit(), text))
    return text

In [15]:
def removeNumbers(text):
    text = ''.join([i for i in text if not i.isdigit()])         
    return text

In [16]:
#Remove unnecessary charracters
def removeUnnecessary(text):
    text=re.sub('\n','',text) # menghapus newline
    text=re.sub('rt','',text) # menghapus kata retweet
    text=re.sub('user','',text) # menghapus kata user
    text=re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))','',text) # menghapus url
    text = re.sub('  +', ' ', text) # menghapus karakter ekstra
    text=re.sub('x.{3} | x.{2}','',text) # menghapus pola emoji 
    return text

In [17]:
#normalize alay
alay=dict(zip(kamus_alay['asli'],kamus_alay['baru']))
def normalizeAlay(text):
    return ' '.join([alay[word] if word in alay else word for word in text.split(' ')])

In [18]:
#Stemming text
def stemWord(x):
    return stemmer.stem(x)

In [19]:
#remove indonesian stopword
def removeStopWord(text):
    text=stopword.remove(text)
    return text

In [20]:
def preProcessing(sentences):
    sentences=lowercase(sentences)
    sentences=removeNonAlphanumeric(sentences)
    sentences=removeUnnecessary(sentences)
    sentences=normalizeAlay(sentences)
    sentences=stemWord(sentences)
    sentences=removeStopWord(sentences)
    return sentences

In [21]:
data['preprocessed']=data['text'].apply(preProcessing)
data

,created at,username,text,preprocessed
0,2022-11-02,rudiartraa,"yaah , udah selesai nih Bjorka Bjorkaan nya ?",selesai nih bjorka bjorkaan
1,2022-11-02,oi_filix315,"Kadang gua mikir sekilas pp gua mirip bjorka, ...",kadang gue pikir kilas pp gue bjorka gue pakai pp
2,2022-11-01,falahsanjaya_,Knp ga ada yg jadi Bjorka Halloween party kema...,bjorka halloween pay kemarin
3,2022-11-01,namanyaajess,"Beritanya taik, mau ferdi sambo, mau bjorka, m...",berita tai ferdi sambo bjorka lapar tren topik...
4,2022-11-01,namakubaim,Apa kabar Bjorka?,kabar bjorka
...,...,...,...,...
19995,2022-09-11,SecretAdmirerOP,hari ini ada 2 hari bersejarah\n9/11\nbjorka s...,sejarah bjorka spill data bunuh munir tau
19996,2022-09-11,yonforger,"lah anjir aku baru tau bjorka tu hacker, kirai...",anjir tau bjorka hacker bubur kacang ijo tukan...
19997,2022-09-11,semangkalucu,kata w mending bjorka ngewarin tiket the link aja,gue mending bjorka ngewarin tiket the link
19998,2022-09-11,choaciao,Bang bjorka bisa ga si hek hati doi biar sama ...,bang bjorka sih hek hati biar ghuweh


In [2]:
import translators as ts

Using state Yogyakarta server backend.


In [3]:
def translate(dataframe,awal,akhir):
    for i in range(awal,akhir):
        try:
            dataframe['english'][i]=ts.google(dataframe['english'][i],from_language='id',to_language='en')
            print("iterasi ke-",i,dataframe['english'][i])
        except Exception as e:
            print("Stop pada iterasi ke-",i)
            print(e)
            return dataframe
    return dataframe

In [28]:
trans=translate(data,2000,4000)
trans

C:\Users\ASUS\AppData\Local\Temp\ipykernel_17832\3045979434.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['english'][i]=ts.google(dataframe['english'][i],from_language='id',to_language='en')


iterasi ke- 0 finished bjorka bjorkaan
iterasi ke- 1 sometimes I think my pp flash is bjorka I use pp
iterasi ke- 2 Bjorka Halloween Pay yesterday
iterasi ke- 3 News Tai Ferdy Sambo Bjorka Hungry Trend Topic Runted Difficult Opponents
iterasi ke- 4 Bjorka news
iterasi ke- 5 Make Daily Social Media Bjorka Headquarters Very complicated Choose the name FC Mang on Jonathan
iterasi ke- 6 Bjorka Kabar
iterasi ke- 7 Instagram I Plow Bjorka Kah
iterasi ke- 8 by the way bjorka news
iterasi ke- 9 Bjorka Need My Account
iterasi ke- 10 Instagram is very news that the wind gossip or naonnn bjorka ngehack Instagram
iterasi ke- 11 Bjorka Me Execution Instagram Instagramdown
iterasi ke- 12 Instagram Bobol Bjorka Password
iterasi ke- 13 Instagram Hacked Bjorka
iterasi ke- 14 lazy to take care of social media Instagramdown confused try bjorka
iterasi ke- 15 anyway Bjorka was viral yesterday
iterasi ke- 16 Wait for the contents of the curhatan I repay sis bjorka huhu
iterasi ke- 17 Sometimes teaching hac

,created at,username,text,preprocessed,english
0,2022-11-02,rudiartraa,"yaah , udah selesai nih Bjorka Bjorkaan nya ?",selesai nih bjorka bjorkaan,finished bjorka bjorkaan
1,2022-11-02,oi_filix315,"Kadang gua mikir sekilas pp gua mirip bjorka, ...",kadang gue pikir kilas pp gue bjorka gue pakai pp,sometimes I think my pp flash is bjorka I use pp
2,2022-11-01,falahsanjaya_,Knp ga ada yg jadi Bjorka Halloween party kema...,bjorka halloween pay kemarin,Bjorka Halloween Pay yesterday
3,2022-11-01,namanyaajess,"Beritanya taik, mau ferdi sambo, mau bjorka, m...",berita tai ferdi sambo bjorka lapar tren topik...,News Tai Ferdy Sambo Bjorka Hungry Trend Topic...
4,2022-11-01,namakubaim,Apa kabar Bjorka?,kabar bjorka,Bjorka news
...,...,...,...,...,...
19995,2022-09-11,SecretAdmirerOP,hari ini ada 2 hari bersejarah\n9/11\nbjorka s...,sejarah bjorka spill data bunuh munir tau,sejarah bjorka spill data bunuh munir tau
19996,2022-09-11,yonforger,"lah anjir aku baru tau bjorka tu hacker, kirai...",anjir tau bjorka hacker bubur kacang ijo tukan...,anjir tau bjorka hacker bubur kacang ijo tukan...
19997,2022-09-11,semangkalucu,kata w mending bjorka ngewarin tiket the link aja,gue mending bjorka ngewarin tiket the link,gue mending bjorka ngewarin tiket the link
19998,2022-09-11,choaciao,Bang bjorka bisa ga si hek hati doi biar sama ...,bang bjorka sih hek hati biar ghuweh,bang bjorka sih hek hati biar ghuweh


In [4]:
trans=pd.read_excel('D:/Skripsi/Skenario Data/Skenario2/DataSementara.xlsx')
del trans['Unnamed: 0']
trans

,created at,username,text,preprocessed,english
0,2022-11-02,rudiartraa,"yaah , udah selesai nih Bjorka Bjorkaan nya ?",selesai nih bjorka bjorkaan,finished bjorka bjorkaan
1,2022-11-02,oi_filix315,"Kadang gua mikir sekilas pp gua mirip bjorka, ...",kadang gue pikir kilas pp gue bjorka gue pakai pp,sometimes I think my pp flash is bjorka I use pp
2,2022-11-01,falahsanjaya_,Knp ga ada yg jadi Bjorka Halloween party kema...,bjorka halloween pay kemarin,Bjorka Halloween Pay yesterday
3,2022-11-01,namanyaajess,"Beritanya taik, mau ferdi sambo, mau bjorka, m...",berita tai ferdi sambo bjorka lapar tren topik...,News Tai Ferdy Sambo Bjorka Hungry Trend Topic...
4,2022-11-01,namakubaim,Apa kabar Bjorka?,kabar bjorka,Bjorka news
...,...,...,...,...,...
19995,2022-09-11,SecretAdmirerOP,hari ini ada 2 hari bersejarah\n9/11\nbjorka s...,sejarah bjorka spill data bunuh munir tau,sejarah bjorka spill data bunuh munir tau
19996,2022-09-11,yonforger,"lah anjir aku baru tau bjorka tu hacker, kirai...",anjir tau bjorka hacker bubur kacang ijo tukan...,anjir tau bjorka hacker bubur kacang ijo tukan...
19997,2022-09-11,semangkalucu,kata w mending bjorka ngewarin tiket the link aja,gue mending bjorka ngewarin tiket the link,gue mending bjorka ngewarin tiket the link
19998,2022-09-11,choaciao,Bang bjorka bisa ga si hek hati doi biar sama ...,bang bjorka sih hek hati biar ghuweh,bang bjorka sih hek hati biar ghuweh


In [10]:
trans=translate(trans,19783,20000)
trans

C:\Users\ASUS\AppData\Local\Temp\ipykernel_15984\2512091061.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe['english'][i]=ts.google(dataframe['english'][i],from_language='id',to_language='en')


iterasi ke- 19783 Bjorka Golek Sek Debt Njug Lost Tanpo Kabar Je
iterasi ke- 19784 Bjorka Read Bejorka Bjorka Sambung Bijorka
iterasi ke- 19785 Bjorka Dalang Hinging MMP Processing the End of a Free Light Law
iterasi ke- 19786 Bjorka hack database bepeka yag wkwk deg
iterasi ke- 19787 Open a busy tweet bjorka wowwwwwww
iterasi ke- 19788 Data leaking children's internship legend tuman bjorka
iterasi ke- 19789 really bjorka hacker orders to save internet access data at any panicked command to be confused
iterasi ke- 19790 Bjorka please leak up
iterasi ke- 19791 Original Fake Fix Bjorka Over the issue haha
iterasi ke- 19792 Momentum Hacker Pei Bjorka Many commands here are the Corrupt Corruption of the Constitutional Protection like the twoar Bjorka Momentum
iterasi ke- 19793 I Hype, Bjorka Drops State Secret Documents
iterasi ke- 19794 Bjorka Indonesia
iterasi ke- 19795 Bjorka is fun
iterasi ke- 19796 Bjorka unloaded roots
iterasi ke- 19797 crazy crazy bjorka page telephone in setting sc

,created at,username,text,preprocessed,english
0,2022-11-02,rudiartraa,"yaah , udah selesai nih Bjorka Bjorkaan nya ?",selesai nih bjorka bjorkaan,finished bjorka bjorkaan
1,2022-11-02,oi_filix315,"Kadang gua mikir sekilas pp gua mirip bjorka, ...",kadang gue pikir kilas pp gue bjorka gue pakai pp,sometimes I think my pp flash is bjorka I use pp
2,2022-11-01,falahsanjaya_,Knp ga ada yg jadi Bjorka Halloween party kema...,bjorka halloween pay kemarin,Bjorka Halloween Pay yesterday
3,2022-11-01,namanyaajess,"Beritanya taik, mau ferdi sambo, mau bjorka, m...",berita tai ferdi sambo bjorka lapar tren topik...,News Tai Ferdy Sambo Bjorka Hungry Trend Topic...
4,2022-11-01,namakubaim,Apa kabar Bjorka?,kabar bjorka,Bjorka news
...,...,...,...,...,...
19995,2022-09-11,SecretAdmirerOP,hari ini ada 2 hari bersejarah\n9/11\nbjorka s...,sejarah bjorka spill data bunuh munir tau,History of Bjorka Spill Munir Kill Data Tau
19996,2022-09-11,yonforger,"lah anjir aku baru tau bjorka tu hacker, kirai...",anjir tau bjorka hacker bubur kacang ijo tukan...,Anjir Tau Bjorka Hacker Green Beans Porridge S...
19997,2022-09-11,semangkalucu,kata w mending bjorka ngewarin tiket the link aja,gue mending bjorka ngewarin tiket the link,I better bjorka to take the link the link
19998,2022-09-11,choaciao,Bang bjorka bisa ga si hek hati doi biar sama ...,bang bjorka sih hek hati biar ghuweh,"bang bjorka, hek heart so ghuweh"


In [11]:
trans.to_excel('D:/Skripsi/Skenario Data/Skenario2/DataSementara.xlsx')

In [12]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
 
# function to print sentiments
# of the sentence.
def sentiment_scores(sentence):
 
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
 
    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)
     
 
    # decide sentiment as positive, negative and neutral
    if sentiment_dict['compound'] >= 0.05 :
        return "Positive"
 
    elif sentiment_dict['compound'] <= - 0.05 :
        return "Negative"
 
    else :
        return "Neutral"

def sentiment_score2(sentence):
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
 
    # polarity_scores method of SentimentIntensityAnalyzer
    # object gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)
     
 
    # decide sentiment as positive, negative and neutral
    if sentiment_dict['compound'] >= 0 :
        return "Positive"
 
    else :
        return "Negative"

In [13]:
trans['sentiment']=trans['english'].apply(sentiment_scores)
trans['sentiment'].value_counts()

Neutral     8906
Positive    6246
Negative    4848
Name: sentiment, dtype: int64

In [14]:
trans

,created at,username,text,preprocessed,english,sentiment
0,2022-11-02,rudiartraa,"yaah , udah selesai nih Bjorka Bjorkaan nya ?",selesai nih bjorka bjorkaan,finished bjorka bjorkaan,Neutral
1,2022-11-02,oi_filix315,"Kadang gua mikir sekilas pp gua mirip bjorka, ...",kadang gue pikir kilas pp gue bjorka gue pakai pp,sometimes I think my pp flash is bjorka I use pp,Neutral
2,2022-11-01,falahsanjaya_,Knp ga ada yg jadi Bjorka Halloween party kema...,bjorka halloween pay kemarin,Bjorka Halloween Pay yesterday,Negative
3,2022-11-01,namanyaajess,"Beritanya taik, mau ferdi sambo, mau bjorka, m...",berita tai ferdi sambo bjorka lapar tren topik...,News Tai Ferdy Sambo Bjorka Hungry Trend Topic...,Negative
4,2022-11-01,namakubaim,Apa kabar Bjorka?,kabar bjorka,Bjorka news,Neutral
...,...,...,...,...,...,...
19995,2022-09-11,SecretAdmirerOP,hari ini ada 2 hari bersejarah\n9/11\nbjorka s...,sejarah bjorka spill data bunuh munir tau,History of Bjorka Spill Munir Kill Data Tau,Negative
19996,2022-09-11,yonforger,"lah anjir aku baru tau bjorka tu hacker, kirai...",anjir tau bjorka hacker bubur kacang ijo tukan...,Anjir Tau Bjorka Hacker Green Beans Porridge S...,Neutral
19997,2022-09-11,semangkalucu,kata w mending bjorka ngewarin tiket the link aja,gue mending bjorka ngewarin tiket the link,I better bjorka to take the link the link,Positive
19998,2022-09-11,choaciao,Bang bjorka bisa ga si hek hati doi biar sama ...,bang bjorka sih hek hati biar ghuweh,"bang bjorka, hek heart so ghuweh",Positive


In [15]:
trans.to_excel('D:/Skripsi/Skenario Data/Skenario2/data_sentiment.xlsx')

In [24]:
data.to_excel('D:/Skripsi/Data/english_sentiment_clean_2polarity.xlsx')